In [63]:
#Importing read-sm-files.py
import ReadSMFIles
from Resource import Resource
from Task import Task

#reading j30.sm/j301_1.sm file
sm_file = ReadSMFIles.SMFileParser.parse_sm_file("j30.sm/j301_1.sm")

print(sm_file[2].successors)


0        [2, 3, 4]
1      [6, 11, 15]
2       [7, 8, 13]
3       [5, 9, 10]
4             [20]
5             [30]
6             [27]
7     [12, 19, 27]
8             [14]
9         [16, 25]
10        [20, 26]
11            [14]
12        [17, 18]
13            [17]
14            [25]
15        [21, 22]
16            [22]
17        [20, 22]
18        [24, 29]
19        [23, 25]
20            [28]
21            [23]
22            [24]
23            [30]
24            [30]
25            [31]
26            [28]
27            [31]
28            [32]
29            [32]
30            [32]
31              []
Name: successors, dtype: object


In [64]:
#Creating resources
r1 = int(sm_file[4].R1[0])
r2 = int(sm_file[4].R2[0])
r3 = int(sm_file[4].R3[0])
r4 = int(sm_file[4].R4[0])

R1 = Resource('R1', r1)
R2 = Resource('R2', r2)
R3 = Resource('R3', r3)
R4 = Resource('R4', r4)

resources = [R1, R2, R3, R4]


In [69]:
#Creating jobs
jobs_enumerate = sm_file[3].jobnr
jobs_duration = sm_file[3].duration
jobs_resources = sm_file[3].resources
jobs_successors = sm_file[2].successors

jobs = [None for _ in jobs_enumerate]


for i in jobs_enumerate:
    jobs[i - 1] = Task(str(i), jobs_duration[i - 1])

for i in range(len(resources)):
    for j in range(len(jobs)):
        jobs[j].add_renewable_resource(resources[i], jobs_resources[j][i])

for i in range(len(jobs)):
    successors = jobs_successors[i]
    for j in successors:
        jobs[i].add_sucessor(jobs[j - 1])
    
for job in jobs:
    print([sucessor.name for sucessor in job.predecessors])

    

    



['2', '3', '4']
['6', '11', '15']
['7', '8', '13']
['5', '9', '10']
['20']
['30']
['27']
['12', '19', '27']
['14']
['16', '25']
['20', '26']
['14']
['17', '18']
['17']
['25']
['21', '22']
['22']
['20', '22']
['24', '29']
['23', '25']
['28']
['23']
['24']
['30']
['30']
['31']
['28']
['31']
['32']
['32']
['32']
[]



Modelagem


In [ ]:
#Dados
n = 0 #numero de trabalhos
m = 0 #número de tipos de recursos



In [ ]:
from Schedule import Schedule
from Task import Task
from Resource import Resource


def print_time_windows_es_ef(schedule: Schedule):
    for task in schedule.tasks:
        print(task.name, [task.earliest_start, task.earliest_finish])

schedule5 = Schedule()

d1 = Task("D1", duration=3)
d2 = Task("D2", duration=4)
d3 = Task("D3", duration=2)
d4 = Task("D4", duration=2)
d5 = Task("D5", duration=1)
d6 = Task("D6", duration=4)
d7 = Task("D7", duration=1)

d3.add_predecessors([d1])
d4.add_predecessors([d2])
d5.add_predecessors([d3])
d6.add_predecessors([d4])
d7.add_predecessors([d1])

r1 = Resource("R1", 4)

d1.add_renewable_resource(r1, 2)
d2.add_renewable_resource(r1, 3)
d3.add_renewable_resource(r1, 4)
d4.add_renewable_resource(r1, 4)
d5.add_renewable_resource(r1, 3)
d6.add_renewable_resource(r1, 2)
d7.add_renewable_resource(r1, 2)

schedule5.add_renewable_resource(r1)

schedule5.add_tasks([d2, d4, d6, d1, d3, d5, d7])

makespan = schedule5.makespan()

print_time_windows_es_ef(schedule5)
print('Schedule 5 Makespan:', makespan)

# def print_task_resources(schedule: Schedule):
#     for task in schedule.tasks:
#         print(task.name, [resource.name for resource in task.renewable_resources], [task.renewable_resources[resource] for resource in task.renewable_resources])

# print_task_resources(schedule5)

Dummy Source [0, 0]
D2 [0, 4]
D4 [4, 6]
D6 [6, 10]
D1 [6, 9]
D3 [10, 12]
D5 [12, 13]
D7 [9, 10]
Dummy Sink [13, 13]
Schedule 5 Makespan: 13
